# 이 폴더의 코드는 https://teddylee777.github.io/huggingface/huggingface-text-classification부터 왔음을 명시합니다.

- 주석 및 코드는 제가 일부만 수정하였을 뿐, 전체적인 흐름은 여기서 가져왔습니다.



In [3]:
import pandas as pd
import numpy as np
import cufflinks as cf
import chart_studio.plotly as py
import plotly.graph_objects as go
import os

import torch

cf.go_offline(connected = True)

In [4]:
import re
 
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData)
 
    return text


In [5]:
import json
import tqdm

In [6]:

from emoji import core

directory = 'C:/Users/USER/Documents/Jupyter_Notebook/KUAIA_Hackerton_3/Total_baseline_data/인플루언서정보_샘플2'

#     print('###################################다음##########################################3')

In [7]:
# lst_score_total = []
# num = 0
# for files in tqdm.tqdm(os.listdir('Total_baseline_data/인플루언서정보_샘플2')[:]):

#     for file in os.listdir('Total_baseline_data/인플루언서정보_샘플2/' + files + '/images' ):
#         if ('jpg' not in file) & ('web' not in file) :


In [8]:
import re
 
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData)
    return text
 
num = 0
lst_index = []
lst_all_normal = []
lst_all_special = []
lst_text = []
for i in os.listdir(directory):
    for j in os.listdir(directory+'/'+i):

        if 'data' in j:


            json_test = pd.read_json(directory+'/'+i + '/' + j)
            lst_index.append(json_test['post_id'].values)
           
            for tt in json_test['text']:
                text =core.replace_emoji(tt)
                text_2 = text.replace('\n','').strip()
                text_3 = [w  for w in text_2.split(' ') if len(w.strip())!=0]

                text_3_1 = [k for k in text_3 if (k[0] != '#') & (k[0] != '@')]
                text_3_2 = []
                for fet in text_3_1:
                    fet = cleanText(fet)
                    if len(fet.strip()) == 0:
                        pass
                    else:
                        text_3_2.append(fet)

                lst_all_normal.append(text_3_2)



                text_4 = [j for j in text_3 if (j[0] == '#') | (j[0] == '@')] 
                lst_all_special.append(text_4)

                lst_text.append([i,' '.join(text_3_2).strip(), text_4])

            num +=1
            
print(num)
            
                




60


In [9]:
np.array(lst_index).ravel()

array([3007458906134705217, 3006010861366948122, 3002176653817258672, ...,
       2984751767654035965, 2983622937128442293, 2983256720106481707],
      dtype=int64)

In [10]:
df_text_data = pd.concat([pd.DataFrame([np.array(lst_index).ravel()]).T, pd.DataFrame(lst_text)],axis =1).copy()



In [11]:
df_text_data

,0,0,1,2
0,3007458906134705217,@hwi____ii,그래서 왜 아직도 난 퇴근을 못했는가,[]
1,3006010861366948122,@hwi____ii,새해 복 많이 받으셔요,[]
2,3002176653817258672,@hwi____ii,오늘은 제 짝꿍 생일이래요오다들 화요팅 하세요오,[]
3,3000980204718489446,@hwi____ii,멜클제품제공 요 귀여운 초록 파우치는 패딩 디자인에 유니크한데160g의 가벼운 무게...,"[@lightpeople_official, #노트북파우치, #노트북파우치추천, #아..."
4,2991478599237586217,@hwi____ii,,"[#제품협찬, @travel_liveincomfort]"
...,...,...,...,...
1795,2986315097145422000,@z___meme,prenda,"[@prendadesigncompany, #프렌다, #스트릿룩, #여자스트릿룩, #..."
1796,2985516610615138008,@z___meme,첫눈이다악수부,"[#subu, #etcseoul, #패딩슬리퍼, #패딩신발, #겨울코디, #여자겨울..."
1797,2984751767654035965,@z___meme,지난 주말에 처음으로 뮤지컬 관람하고왔어요 배우님들 연기하시고 노래하시는 거 너무 ...,"[#광고, #용의자x의헌신, #용의자x의헌신뮤지컬, #뮤지컬, #뮤지컬추천, #뮤지..."
1798,2983622937128442293,@z___meme,곧 다가오는 연말 건조한 피부를 위한 추천 정품은 아이소이 지우개세럼 흔적세럼 그리...,"[#유료광고잡티세럼,, #잡티세럼, #올리브영에센스, #연말선물]"


# 여기부터 모델

In [12]:
# 한글 자연어 처리 데이터셋
from Korpora import Korpora

# 토크나이저 관련 경고 무시하기 위하여 설정
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

# device 지정
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

corpus = Korpora.load("nsmc")

사용 디바이스: cuda:1

    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\USER\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed 

In [13]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader


class TokenDataset(Dataset):
  
    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe        
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['document']
        label = self.data.iloc[idx]['label']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [14]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [15]:
df_test = pd.read_excel('First_to_Second_page.xlsx')

In [16]:
df_test_part = df_test[['style','comment','tag']].copy()

In [17]:
from sklearn.model_selection import train_test_split


t1,t2 =train_test_split(range(df_test_part.shape[0]),stratify = df_test_part['style'],test_size = 0.2)



In [18]:
from sklearn.preprocessing import LabelEncoder
items=['아메리칸 캐주얼', '캐주얼', '댄디', '포멀', '걸리시', '골프', '홈웨어', '레트로', '로맨틱',
       '스포츠', '스트릿', '시크']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
print(labels)
len(labels)

[ 8  9  2 10  0  1 11  3  4  6  5  7]


12

In [19]:
def manipulate(row):
    row['style'] = encoder.transform([row['style']])[0]
    return row

In [20]:
df_test_part_2 = df_test_part.apply(manipulate,axis=1).copy()

In [21]:
df_test_part_2 = df_test_part.copy()

In [22]:
df_test_part_2.reset_index(drop=True, inplace = True)

In [23]:
train = df_test_part_2.loc[t1,['style','comment']]
test = df_test_part_2.loc[t2,['style','comment']]

In [24]:
train.columns = ['label', 'document']
test.columns =  ['label', 'document']

In [25]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'
tokenizer_pretrained = CHECKPOINT_NAME

In [26]:
# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=4,shuffle=True, num_workers=0)

In [27]:
device = "cuda:0"

In [28]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

tensor([ 0,  2,  6, 11], device='cuda:0', dtype=torch.int32)

In [29]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)


Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
import torch.nn as nn

In [31]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 12)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [32]:
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CustomBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [33]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.type(torch.LongTensor).to(device)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)
        
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()
        
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)
        
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [34]:
def model_evaluate(model, data_loader, loss_fn, device):
    # model.eval()은 모델을 평가모드로 설정을 바꾸어 줍니다. 
    # dropout과 같은 layer의 역할 변경을 위하여 evaluation 진행시 꼭 필요한 절차 입니다.
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 하기 위하여 반드시 필요합니다.
    with torch.no_grad():
        # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
        corr = 0
        running_loss = 0
        
        # 배치별 evaluation을 진행합니다.
        for inputs, labels in data_loader:
            # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.type(torch.LongTensor).to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [35]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = torch.optim.Adam(bert.parameters(), lr=1e-5)

In [35]:
# 최대 Epoch을 지정합니다.
num_epochs = 30

# checkpoint로 저장할 모델의 이름을 정의 합니다.
model_name = 'bert-kor-base'

min_loss = np.inf

# Epoch 별 훈련 및 검증을 수행합니다.
for epoch in range(num_epochs):
    # Model Training
    # 훈련 손실과 정확도를 반환 받습니다.
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

training loss: 1.50192, training accuracy: 0.89412: 100%|███████████████████████| 1396/1396 [12:00<00:00,  1.94batch/s]


[INFO] val_loss has been improved from inf to 0.01057. Saving Model!
epoch 01, loss: 0.37561, acc: 0.89412, val_loss: 0.01057, val_accuracy: 0.99857


training loss: 0.03432, training accuracy: 0.99910: 100%|███████████████████████| 1396/1396 [12:11<00:00,  1.91batch/s]


[INFO] val_loss has been improved from 0.01057 to 0.00828. Saving Model!
epoch 02, loss: 0.00858, acc: 0.99910, val_loss: 0.00828, val_accuracy: 0.99857


training loss: 0.00808, training accuracy: 1.00000:   6%|█▌                       | 85/1396 [00:45<11:47,  1.85batch/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-48edc67dcad5>", line 13, in <module>
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)
  File "<ipython-input-33-a174df462aa5>", line 31, in model_train
    loss.backward()
  File "C:\Users\USER\Anaconda3\lib\site-packages\torch\_tensor.py", line 489, in backward
    self, gradient, retain_graph, create_graph, inputs=inputs
  File "C:\Users\USER\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 199, in backward
    allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-48edc67dcad5>", line 13, in <module>
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)
  File "<ipython-input-33-a174df462aa5>", line 31, in model_train
    loss.backward()
  File "C:\Users\USER\Anaconda3\lib\site-packages\torch\_tensor.py", line 489, in backward
    self, gradient, retain_graph, create_graph, inputs=inputs
  File "C:\Users\USER\Anaconda3\lib\site-packages\torch\autograd\__init__.py", line 199, in backward
    allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", 

TypeError: object of type 'NoneType' has no len()